# 0: imports

In [1]:
import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


# 1: load data

### Read file

In [2]:
with open("wonderland.txt","r") as file:
    raw_text = file.read().lower()

### Encode chars

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144435
Total Vocab:  48


# 2: encode data

In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  144335


### encode x data

In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)

### one hot encode y variable

In [7]:
y = np_utils.to_categorical(dataY)

# 3: model

### construct

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

### checkpointing

In [9]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

### train

In [32]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list, verbose=0)

# 4: load best model

In [12]:
# load the network weights
filename = "weights-improvement-29-1.6748.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

### reverse mapping

In [12]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

# 5: generate text

### Random seed

In [25]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters

Seed:
"  in the sand with wooden spades, then a row
of lodging houses, and behind them a railway station.) h "


### Generate text

In [26]:
generated_string = ""

for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    generated_string += result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")


Done.


In [27]:
print(generated_string)

n she sas ooo of the torde aerer that she was sorr that the had so her an thls wiuh the hooke of the court, and whs so tere the dorrt to the kant aealr, and the shrteht th the coor and sookdd a little brat in the wordd he thth the gors, and the queen sat al torer of the court, and the horp and boowdd a cottoeng to ceon in the simd, and the qhreg harderer oh the thate whs oo her ferting to tee ht whon his fend hn the way th they wat to tery torer of the hooket of the hoose of the court, 'and the murer oo that it touhd ho wou dane ii the sigee the hadee of the bourte oh the hoose, and the hors ao spld as she wosld got then the houke, and the queen saide  'to the codhess wotld ae ooe cadl an she could, woth the had sote the had sooere the had so bell to thy th tenen dnenged to she thnte then sas soine the hadle hn thet would hareen, and the queen saide in the siade 'shi had so thil to shyh the hotth oo thet whuh ani thry histle bertere got hn the wind whyh ani the was gowning and the had 

# 6: bigger network

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list,verbose=0)

InternalError: Blas GEMM launch failed : a.shape=(6400, 1), b.shape=(1, 256), m=6400, n=256, k=1
	 [[Node: lstm_2/MatMul_2 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](lstm_2/Reshape, lstm_2/strided_slice_2)]]
	 [[Node: training/Adam/gradients/AddN/_59 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1917_training/Adam/gradients/AddN", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'lstm_2/MatMul_2', defined at:
  File "C:\Users\peter\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\peter\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\peter\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_n

In [10]:
# load the network weights
filename = "weights-improvement-47-1.2633-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters

Seed:
" fe. indeed, she had quite a long argument with the
lory, who at last turned sulky, and would only sa "


In [21]:
generated_string = ""

for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    generated_string += result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")


Done.


In [22]:
generated_string

"y the book, and the peabling of the jurymen of her ereature, and she could not look the book, and the pteen was see and ar hard as she could, and the jory that the had fou to tee it as the pueen, and the gryphon had jook at the rabbit seimed, and the thing ieard the had fou to tee it as the way of semlo of the sorm of the same with the room, and she was see and sreeenly as the couldu. \n'then it wasn't gite that dirictencs to the thing i can the daysherl raie!' said the daterpillar.\n\n'well, i mever heard of your have out of the siinge with the cance?\n   yiile the dormouse surped to the kobster. \n    and mowhhd the sea of the ereature, and the srpebless was the rabe then it was the book, and the poor little thing had found herself tp and said, 'and then the book was the rame thing as the way of mearing and mobkes the bourt, and the thing ier head something about it, and the daterpillar conling she had found herself up and said, 'and then the book was the rame thing as the court, an